In [ ]:
import pandas as pd

csv_files = [
    "/content/api_data_aadhar_enrolment_0_500000.csv",
    "/content/api_data_aadhar_enrolment_500000_1000000.csv",
    "/content/api_data_aadhar_enrolment_1000000_1006029.csv"
]
combined_df = pd.concat(
    [pd.read_csv(file) for file in csv_files],
    ignore_index=True
)
combined_df = combined_df[
    [
        "date",
        "state",
        "district",
        "pincode",
        "age_0_5",
        "age_5_17",
        "age_18_greater"
    ]
]
combined_df.to_csv("enrollment.csv", index=False)
print("✅ 4 CSV files combined successfully into combined_dataset.csv")

✅ 4 CSV files combined successfully into combined_dataset.csv


In [ ]:
import pandas as pd

df_bio = pd.read_csv('/content/unified_bio_data_final.csv')
df_demo = pd.read_csv('/content/unified_demo_data_final.csv')
df_age = pd.read_csv('/content/unified_enrol_data_final.csv')
for df in [df_bio, df_demo, df_age]:
    df['date'] = pd.to_datetime(df['date'], dayfirst=True, format='mixed')
    df['pincode'] = df['pincode'].astype(str)
    df['state'] = df['state'].str.lower().str.strip()
    df['district'] = df['district'].str.lower().str.strip()
merge_keys = ['date', 'state', 'district', 'pincode']
merged_df = pd.merge(df_bio, df_demo, on=merge_keys, how='outer')
final_df = pd.merge(merged_df, df_age, on=merge_keys, how='outer')
metric_columns = [
    'bio_age_5_17', 'bio_age_17_',
    'demo_age_5_17', 'demo_age_17_',
    'age_0_5', 'age_5_17', 'age_18_greater'
]
final_df[metric_columns] = final_df[metric_columns].fillna(0)
final_df = final_df.sort_values(by=['date', 'state', 'district'])
print("Merged Dataset Head:")
print(final_df.head())
final_df.to_csv('unified_enrolment_data.csv', index=False)
print(f"Total rows in combined dataset: {len(final_df)}")

Merged Dataset Head:
        date                        state  district pincode  bio_age_5_17  \
0 2025-03-01  andaman and nicobar islands  andamans  744101          16.0   
1 2025-03-01  andaman and nicobar islands   nicobar  744301         101.0   
2 2025-03-01  andaman and nicobar islands   nicobar  744301         101.0   
3 2025-03-01  andaman and nicobar islands   nicobar  744302          15.0   
4 2025-03-01  andaman and nicobar islands   nicobar  744303          46.0   

   bio_age_17_  demo_age_5_17  demo_age_17_  age_0_5  age_5_17  age_18_greater  
0        193.0            0.0           0.0      0.0       0.0             0.0  
1         48.0           16.0         180.0      0.0       0.0             0.0  
2         48.0           16.0         180.0      0.0       0.0             0.0  
3         12.0            0.0           0.0      0.0       0.0             0.0  
4         27.0            0.0           0.0      0.0       0.0             0.0  
Total rows in combined dataset

In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

df = pd.read_csv("/content/biometric.csv")

def normalize(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = text.replace("&", "and")
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["state_norm"] = df["state"].apply(normalize)

OFFICIAL_STATES = [
    "Andaman and Nicobar Islands",
    "Andhra Pradesh",
    "Arunachal Pradesh",
    "Assam",
    "Bihar",
    "Chandigarh",
    "Chhattisgarh",
    "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi",
    "Goa",
    "Gujarat",
    "Haryana",
    "Himachal Pradesh",
    "Jammu and Kashmir",
    "Jharkhand",
    "Karnataka",
    "Kerala",
    "Ladakh",
    "Lakshadweep",
    "Madhya Pradesh",
    "Maharashtra",
    "Manipur",
    "Meghalaya",
    "Mizoram",
    "Nagaland",
    "Odisha",
    "Puducherry",
    "Punjab",
    "Rajasthan",
    "Sikkim",
    "Tamil Nadu",
    "Telangana",
    "Tripura",
    "Uttar Pradesh",
    "Uttarakhand",
    "West Bengal"
]

OFFICIAL_NORM = {normalize(s): s for s in OFFICIAL_STATES}

def predict_state(value, threshold=80):
    if not value:
        return None

    match, score, _ = process.extractOne(
        value,
        OFFICIAL_NORM.keys(),
        scorer=fuzz.token_sort_ratio
    )

    if score >= threshold:
        return OFFICIAL_NORM[match]
    return None

df["state_predicted"] = df["state_norm"].apply(predict_state)

df_clean = df[df["state_predicted"].notna()].copy()

df_clean["state"] = df_clean["state_predicted"]

df_clean.drop(columns=["state_norm", "state_predicted"], inplace=True)

output_path = "/content/unified_bio_data_final.csv"
df_clean.to_csv(output_path, index=False)

print("✅ Cleaning complete")
print("Rows before:", len(df))
print("Rows after :", len(df_clean))
print("Unique states:", df_clean["state"].nunique())
print(f"📁 Saved to: {output_path}")


✅ Cleaning complete
Rows before: 1861108
Rows after : 1844512
Unique states: 36
📁 Saved to: /content/unified_bio_data_final.csv


In [ ]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

df = pd.read_csv("/content/demograph.csv")

def normalize(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = text.replace("&", "and")
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["state_norm"] = df["state"].apply(normalize)

OFFICIAL_STATES = [
    "Andaman and Nicobar Islands",
    "Andhra Pradesh",
    "Arunachal Pradesh",
    "Assam",
    "Bihar",
    "Chandigarh",
    "Chhattisgarh",
    "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi",
    "Goa",
    "Gujarat",
    "Haryana",
    "Himachal Pradesh",
    "Jammu and Kashmir",
    "Jharkhand",
    "Karnataka",
    "Kerala",
    "Ladakh",
    "Lakshadweep",
    "Madhya Pradesh",
    "Maharashtra",
    "Manipur",
    "Meghalaya",
    "Mizoram",
    "Nagaland",
    "Odisha",
    "Puducherry",
    "Punjab",
    "Rajasthan",
    "Sikkim",
    "Tamil Nadu",
    "Telangana",
    "Tripura",
    "Uttar Pradesh",
    "Uttarakhand",
    "West Bengal"
]

OFFICIAL_NORM = {normalize(s): s for s in OFFICIAL_STATES}

def predict_state(value, threshold=80):
    if not value:
        return None

    match, score, _ = process.extractOne(
        value,
        OFFICIAL_NORM.keys(),
        scorer=fuzz.token_sort_ratio
    )

    if score >= threshold:
        return OFFICIAL_NORM[match]
    return None

df["state_predicted"] = df["state_norm"].apply(predict_state)

df_clean = df[df["state_predicted"].notna()].copy()

df_clean["state"] = df_clean["state_predicted"]

df_clean.drop(columns=["state_norm", "state_predicted"], inplace=True)

output_path = "/content/unified_demo_data_final.csv"
df_clean.to_csv(output_path, index=False)

print("✅ Cleaning complete")
print("Rows before:", len(df))
print("Rows after :", len(df_clean))
print("Unique states:", df_clean["state"].nunique())
print(f"📁 Saved to: {output_path}")


✅ Cleaning complete
Rows before: 2071700
Rows after : 2054530
Unique states: 36
📁 Saved to: /content/unified_demo_data_final.csv


In [ ]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

df = pd.read_csv("/content/enrollment.csv")

def normalize(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = text.replace("&", "and")
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["state_norm"] = df["state"].apply(normalize)

OFFICIAL_STATES = [
    "Andaman and Nicobar Islands",
    "Andhra Pradesh",
    "Arunachal Pradesh",
    "Assam",
    "Bihar",
    "Chandigarh",
    "Chhattisgarh",
    "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi",
    "Goa",
    "Gujarat",
    "Haryana",
    "Himachal Pradesh",
    "Jammu and Kashmir",
    "Jharkhand",
    "Karnataka",
    "Kerala",
    "Ladakh",
    "Lakshadweep",
    "Madhya Pradesh",
    "Maharashtra",
    "Manipur",
    "Meghalaya",
    "Mizoram",
    "Nagaland",
    "Odisha",
    "Puducherry",
    "Punjab",
    "Rajasthan",
    "Sikkim",
    "Tamil Nadu",
    "Telangana",
    "Tripura",
    "Uttar Pradesh",
    "Uttarakhand",
    "West Bengal"
]

OFFICIAL_NORM = {normalize(s): s for s in OFFICIAL_STATES}

def predict_state(value, threshold=80):
    if not value:
        return None

    match, score, _ = process.extractOne(
        value,
        OFFICIAL_NORM.keys(),
        scorer=fuzz.token_sort_ratio
    )

    if score >= threshold:
        return OFFICIAL_NORM[match]
    return None

df["state_predicted"] = df["state_norm"].apply(predict_state)

df_clean = df[df["state_predicted"].notna()].copy()

df_clean["state"] = df_clean["state_predicted"]

df_clean.drop(columns=["state_norm", "state_predicted"], inplace=True)

output_path = "/content/unified_enrol_data_final.csv"
df_clean.to_csv(output_path, index=False)

print(" Cleaning complete")
print("Rows before:", len(df))
print("Rows after :", len(df_clean))
print("Unique states:", df_clean["state"].nunique())
print(f" Saved to: {output_path}")


✅ Cleaning complete
Rows before: 1006029
Rows after : 1001566
Unique states: 36
📁 Saved to: /content/unified_enrol_data_final.csv
